In [3]:
import pandas as pd
import numpy as np
import plotly_express as px
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression


In [2]:
wal_df = pd.read_csv('C:/Users/romai/Documents/Data_Science_Courses/Fullstack/Jedha_Fullstack_Data_Science/Project/Walmart/src/Walmart_Store_sales.csv')

# Part 1 : EDA

In [3]:
wal_df.head(10)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092
5,4.0,28-05-2010,1857533.70,0.0,NaN,2.756,126.160226,7.896
6,15.0,03-06-2011,695396.19,0.0,69.80,4.069,134.855161,7.658
7,20.0,03-02-2012,2203523.20,0.0,39.93,3.617,213.023622,6.961
8,14.0,10-12-2010,2600519.26,0.0,30.54,3.109,NaN,NaN
9,3.0,NaN,418925.47,0.0,60.12,3.555,224.132020,6.833


In [4]:
wal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         150 non-null    float64
 1   Date          132 non-null    object 
 2   Weekly_Sales  136 non-null    float64
 3   Holiday_Flag  138 non-null    float64
 4   Temperature   132 non-null    float64
 5   Fuel_Price    136 non-null    float64
 6   CPI           138 non-null    float64
 7   Unemployment  135 non-null    float64
dtypes: float64(7), object(1)
memory usage: 9.5+ KB


In [5]:
(wal_df.isna().sum() / wal_df.shape[0]).apply(lambda x: f'{np.around(x*100, decimals=2)} %')

Store            0.0 %
Date            12.0 %
Weekly_Sales    9.33 %
Holiday_Flag     8.0 %
Temperature     12.0 %
Fuel_Price      9.33 %
CPI              8.0 %
Unemployment    10.0 %
dtype: object

In [6]:
wal_df['Holiday_Flag'].value_counts()

Holiday_Flag
0.0    127
1.0     11
Name: count, dtype: int64

Date is of type object and not datetime, holiday_flag is a bool

It seems there are missing values in all columns (expect 'Store' -> the id)

In [7]:
# Dayfirst = True ici care le format des dates est en europeen dd/mm/year, contre-intruitif car la data concerne un shop americain
wal_df['Date']=pd.to_datetime(wal_df['Date'], dayfirst=True)

We do not need more than year and month to predict weekly sales

In [8]:
wal_df['Month'] = wal_df['Date'].dt.month
wal_df['Year'] = wal_df['Date'].dt.year

In [9]:
wal_df['Year'].value_counts()

Year
2010.0    54
2011.0    43
2012.0    35
Name: count, dtype: int64

In [10]:
px.histogram(wal_df, 'Month', 'Weekly_Sales')

On pourrait encoder les dates dans une deuxieme colonne en utilisant les règles de trigo (sinus et cosinus) comme ça on peut les placer sur un cercle et le modèle comprend que c'est ciruclaire, que le 1 vient après le 12, etc...

In [11]:
wal_df.head(5)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858,2.0,2011.0
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470,3.0,2011.0
2,17.0,2012-07-27,NaN,0.0,NaN,NaN,130.719581,5.936,7.0,2012.0
3,11.0,NaT,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN
4,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092,5.0,2010.0


## The target is weekly sales

In [12]:
# The target does not seem to have outliers, 10% of NA
px.bar(wal_df['Weekly_Sales'])

We can drop the lines with missing weekly sales values as it represents 'only' 10%

In [13]:
condition = wal_df['Weekly_Sales'].isna() == False

# Ou j'aurais pu garder en == True et faire un wal_df = wal_df[~condition] pour appliquer l'inverse du mask

In [14]:
wal_df = wal_df[condition]
(wal_df.isna().sum() / wal_df.shape[0]).apply(lambda x: f'{np.around(x*100, decimals=2)} %')

Store             0.0 %
Date            13.24 %
Weekly_Sales      0.0 %
Holiday_Flag     8.09 %
Temperature     11.03 %
Fuel_Price       8.82 %
CPI              8.09 %
Unemployment    10.29 %
Month           13.24 %
Year            13.24 %
dtype: object

In [15]:
len(wal_df)

136

Plot all columns to check for outliers

In [16]:
px.histogram(wal_df, y='Unemployment')

In [17]:
px.histogram(wal_df, y='Fuel_Price')

In [18]:
px.histogram(wal_df, y='CPI')

In [19]:
px.histogram(wal_df, y='Temperature')

In [20]:
# for c in wal_df.columns:
#     fig = px.scatter(wal_df, x=wal_df.index, y=c, title=f'{c}')
#     fig.show()

We can see that Unemployment (and CPI?) at least has some outliers

Remove the outliers according to the 3 sigmas rule

Appliquer le masque des 3 sigmas en boucle sur le dataframe sans l'écraser

afficher le total des valeurs apres l'application du masque

si cette valewur est > 0, c'est qu'il ya des outliers

I am looking for outliers in Temperature, Fuel price, CPI and Unemployment and I am dropping the four first columns :

1) Store because it is an id
2) Date becasue it is a date
3) Weekly sales because it is my target
4) Holiday flag because it is a boolean

In [21]:
wal_df_trim = wal_df.drop(wal_df[wal_df.columns[:4]], axis=1)
wal_df_trim.head(5)

,Temperature,Fuel_Price,CPI,Unemployment,Month,Year
0,59.61,3.045,214.777523,6.858,2.0,2011.0
1,42.38,3.435,128.616064,7.470,3.0,2011.0
3,84.57,NaN,214.556497,7.346,NaN,NaN
4,78.89,2.759,212.412888,7.092,5.0,2010.0
5,NaN,2.756,126.160226,7.896,5.0,2010.0


## Reprendre ça

Voir le notebook logistic regression

conditoin devrai etre apres le train test split pour filtre test en fonction de la condition de train. Pas sur que besoin ici 

In [22]:
#filter all thédfg using only one column 

outlier_unemp_condition = (wal_df['Unemployment'] < wal_df['Unemployment'].mean() + 3* wal_df['Unemployment'].std()) & (wal_df['Unemployment'] > wal_df['Unemployment'].mean() - 3* wal_df['Unemployment'].std())


In [23]:
test_df = wal_df[outlier_unemp_condition]
test_df

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858,2.0,2011.0
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470,3.0,2011.0
3,11.0,NaT,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN
4,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092,5.0,2010.0
5,4.0,2010-05-28,1857533.70,0.0,NaN,2.756,126.160226,7.896,5.0,2010.0
...,...,...,...,...,...,...,...,...,...,...
142,3.0,2011-10-07,403342.40,0.0,75.54,3.285,NaN,7.197,10.0,2011.0
143,3.0,2010-06-04,396968.80,0.0,78.53,2.705,214.495838,7.343,6.0,2010.0
144,3.0,2012-10-19,424513.08,0.0,73.44,3.594,226.968844,6.034,10.0,2012.0
145,14.0,2010-06-18,2248645.59,0.0,72.62,2.780,182.442420,8.899,6.0,2010.0


In [24]:
px.histogram(test_df, y='Unemployment')

The filter is working, so now I need to iteratively apply it to all the dataframe but by basing the filter on specific columns

In [25]:
len(wal_df)

136

In [26]:
# The 3sigmas rule :

#Should be done on the train 
# outlier_condition = (wal_df_trim[c] < (wal_df_trim[c].mean() + 3* wal_df_trim[c].std())) & (wal_df_trim[c] > (wal_df_trim[c].mean() - 3* wal_df_trim[c].std()))

columns = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']

for column in columns:
    outlier_condition = (wal_df[column] < wal_df[column].mean() + 3* wal_df[column].std()) & (wal_df[column] > wal_df[column].mean() - 3* wal_df[column].std())
    filtered_wal_df = wal_df[outlier_condition]


# def detect_outlier(df, columns):
#     outlier_condition = df[columns] < df[columns].mean() + 3* df[columns].std()
#     to_drop = df[~outlier_condition]
#     df = df.drop(to_drop)
#     return df

In [27]:
'''to count the number of outliers per categories, to fix since I defined the detect_outlier function above'''

wal_df_trim = wal_df.drop(wal_df[wal_df.columns[:4]], axis=1)
wal_df_trim.head(5)

n_outliers = {}
for c in wal_df_trim.columns:
    temp = wal_df_trim[c][~outlier_condition]
    n_outliers[c] = temp.value_counts().sum()
n_outliers

{'Temperature': 15,
 'Fuel_Price': 18,
 'CPI': 17,
 'Unemployment': 5,
 'Month': 16,
 'Year': 16}

In [28]:
len(filtered_wal_df)

117

In [29]:
px.histogram(filtered_wal_df, y='Unemployment')

To simplify the worflow, I overide the old wal_df with the filtered_wal_df

In [30]:
wal_df = filtered_wal_df

I am filling a dictionnary with the sum of occurences of outliers contained in each column from the filtered dataframe :

Now we drop all the lines from the complete dataframe that contains outliers in each of these columns

In [31]:
len(wal_df)

117

In [32]:
wal_df = wal_df[outlier_condition]
wal_df.reset_index(drop=[0])

## Dropped 18 lines

C:\Users\romai\AppData\Local\Temp\ipykernel_24880\858267249.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858,2.0,2011.0
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470,3.0,2011.0
2,11.0,NaT,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN
3,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092,5.0,2010.0
4,4.0,2010-05-28,1857533.70,0.0,NaN,2.756,126.160226,7.896,5.0,2010.0
...,...,...,...,...,...,...,...,...,...,...
112,3.0,2011-10-07,403342.40,0.0,75.54,3.285,NaN,7.197,10.0,2011.0
113,3.0,2010-06-04,396968.80,0.0,78.53,2.705,214.495838,7.343,6.0,2010.0
114,3.0,2012-10-19,424513.08,0.0,73.44,3.594,226.968844,6.034,10.0,2012.0
115,14.0,2010-06-18,2248645.59,0.0,72.62,2.780,182.442420,8.899,6.0,2010.0


In [33]:
for c in wal_df.columns:
    fig = px.scatter(wal_df, x=wal_df.index, y=c, title=f'{c}')
    fig.show()

The dataframe is now clean and ready for model learning, recap : 

1) Too many NA removed
2) Date changed to datetime format
3) Lines with outliers in the four relevant features were removed

In [34]:
# Save the filtered dataframe

# wal_df.to_csv('./src/Filtered_Walmart_data_wdates.csv')

# Part 2 : Baseline model

In [4]:
wal_df = pd.read_csv('./src/Filtered_Walmart_data_wdates.csv', index_col = [0])
wal_df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858,2.0,2011.0
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470,3.0,2011.0
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN
4,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092,5.0,2010.0
5,4.0,2010-05-28,1857533.70,0.0,NaN,2.756,126.160226,7.896,5.0,2010.0


Dropping Dates as we already extracted Year and Month

In [5]:
wal_df.drop('Date', axis=1, inplace=True)

Feature engineering

In [6]:
features_list = wal_df.drop('Weekly_Sales', axis=1).columns.to_list() 
target_name = 'Weekly_Sales'

X = wal_df[features_list]
y = wal_df[target_name]

In [7]:
X.head(0)

,Store,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)  # No need for stratify here

Pipeline

- numerical variables =

    -- continuous = Temperature, Fuel_price, CPI, Unemployment, Month, Year, Weekly_Sales (but target so not included)

- categorical variables =

    -- Store and Holiday_Flag (boolean)

No need for encoding we need scaling and imputer for missing numerical values estimate ('mean')


In [9]:
X.head(0)

,Store,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Month,Year


In [10]:
# I am not automatizing numerical/categorical attribution because my categorical variables are in float
numerical_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Month', 'Year']
categorical_features = ['Holiday_Flag', 'Store']

I define my pipeline for estimating missing numerical values by mean stategy, for scaling numerical values, for encoding month, year and holiday_flag and for estimating missing categortical values based on the most frequent strategy

In [11]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(
    steps=[
    ('encoder', OneHotEncoder(drop='first')),
    ('imputer', SimpleImputer(strategy='most_frequent'))
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

Now we can train the model. We are predicting a continuous numerical value so I am using a linear regression.

In [12]:
lr = LinearRegression()

In [13]:
lr.fit(X_train,y_train)

LinearRegression()

Showing the model accuracy

In [14]:
print('Train score is : {:.2f}'.format(lr.score(X_train, y_train)))
print('Test score is : {:.2f}'.format(lr.score(X_test, y_test)))

Train score is : 0.97
Test score is : 0.94


In [15]:
y_train_pred = lr.predict(X_train)
y_train

17     1205307.50
83      607475.44
37      988157.72
27      435397.19
65     1714309.90
          ...    
132    2036231.39
16     1997397.63
113    1811606.21
59     2020550.99
127     614253.33
Name: Weekly_Sales, Length: 93, dtype: float64

In [16]:
y_train_df = pd.DataFrame(y_train)
y_train_pred_df = pd.DataFrame(y_train_pred)

y_df = pd.concat([y_train_df, y_train_pred_df], ignore_index=True, axis=1)
y_df.rename(columns = {0: 'y_train', 1: 'y_train_pred'})

,y_train,y_train_pred
17,1205307.50,2.078008e+06
83,607475.44,3.867943e+05
37,988157.72,8.184476e+05
27,435397.19,2.495684e+06
65,1714309.90,2.140582e+06
...,...,...
82,NaN,3.705963e+05
84,NaN,1.327518e+06
87,NaN,8.176372e+05
91,NaN,2.052575e+06


To refine, but there is idea

In [17]:
px.histogram(y_df, opacity=0.5)

Good scores for a baseline model but there is some overfitting

Extract coeff to filter for best features

In [18]:
coef = lr.coef_.tolist()

I am filling a dictionnary with my coef per feature

In [19]:
coef_features = {}
coef_features = {k: v for k, v in zip(features_list, coef)}
coef_features

{'Store': -58074.59061947372,
 'Holiday_Flag': -37567.251642572446,
 'Temperature': 25317.170312669805,
 'Fuel_Price': -19765.27869691842,
 'CPI': 75785.89834320401,
 'Unemployment': 8099.028889950512,
 'Month': -42396.757878205695,
 'Year': -88670.9700929581}

Now I am sorting those coef in ascending order (in absolute values as I am not interested in polarity, only intensity)

In [20]:
coef_df = pd.DataFrame(coef_features.items(), columns=['Features', 'Coeff'])
coef_df['Coeff'] = abs(coef_df['Coeff'])
coef_df = coef_df.sort_values(by='Coeff', ascending=True)
coef_df

,Features,Coeff
5,Unemployment,8099.028890
3,Fuel_Price,19765.278697
2,Temperature,25317.170313
1,Holiday_Flag,37567.251643
6,Month,42396.757878
0,Store,58074.590619
4,CPI,75785.898343
7,Year,88670.970093


In [21]:
px.bar(coef_df, 'Features', 'Coeff', title='The most important features')

So it seems Year is the best feature to predict the amount of weekly sales

## Part 3, fighting overfitting

I could use Lasso for feature selection (since we have multiple features) and then Ridge to penalize the model in order to fight overfitting.

In [22]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV

Ridge

In [23]:
ridge = Ridge()
params = {
    'alpha': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100] 
}

In [24]:
gridsearch_ridge = GridSearchCV(ridge, params, cv=5)

Train

In [25]:
gridsearch_ridge.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100]})

In [26]:
gridsearch_ridge.best_estimator_.predict(X_train)

array([1255663.90173069,  615685.70804393,  985005.38725111,
        497478.27232151, 1827919.95407389,  987884.87787262,
        387276.08831766, 1906685.66152492, 1729356.37070262,
        434199.04309497, 1937264.69401096, 1763715.8069568 ,
       1222911.28849294,  417265.31766749,  552369.34235604,
        271002.0120717 , 2038899.36388305, 2076612.73703192,
       1853508.09447887,  633120.30414054, 1008652.72114431,
        659249.74036209, 1834715.5468465 ,  582852.88312738,
       2075108.37596757, 1616689.86312578, 1015991.32129549,
       2468159.18945725,  419405.1168832 ,  462322.34314982,
       1873290.67082136, 2010086.32861566,  610657.14697098,
       1164110.50283524, 1585125.18777704,  613577.14402129,
        876343.0915456 ,  835462.21162968,  944879.8816669 ,
        917968.78400202, 1509591.6558235 , 2081216.64526277,
       1584429.38820605,  620903.413927  , 1479160.98122675,
       2111994.47810016, 1862312.96561476, 2040025.03445296,
       1392293.8193734 ,

In [27]:
print("Best hyperparameters : ", gridsearch_ridge.best_params_)
print("Best R2 score : ", gridsearch_ridge.best_score_)

Best hyperparameters :  {'alpha': 0.05}
Best R2 score :  0.9184360309920081


Lasso

In [28]:
lasso = Lasso()

params = {
    'alpha': [1, 2, 3, 5, 10, 20, 30] 
}

In [30]:
gridsearch_lasso = GridSearchCV(lasso, params, cv=5)

In [31]:
gridsearch_lasso.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [1, 2, 3, 5, 10, 20, 30]})

In [32]:
gridsearch_lasso.best_estimator_.predict(X_train)

array([1246115.51983994,  608237.01241214,  979831.88662894,
        490061.09559904, 1827025.40621029,  985868.42757434,
        370972.31151158, 1910022.23650756, 1721932.96470689,
        414501.23251767, 1950744.4494411 , 1797697.19805741,
       1218666.72928668,  403161.76033695,  550358.32323027,
        267010.26950446, 2031724.63207542, 2077477.72330625,
       1862326.84984826,  628989.60590214, 1006875.9616235 ,
        639555.1843971 , 1843883.63006909,  572433.58545338,
       2083685.93640799, 1654722.62567916, 1019031.10338735,
       2493414.18067798,  420703.58371338,  448575.98043806,
       1866132.85353296, 2017638.17208847,  594769.13106657,
       1170317.88078516, 1584210.43188023,  608259.76093734,
        862575.72935597,  819737.20862337,  940485.49743306,
        913426.70737802, 1504912.41910623, 2083032.66616417,
       1586543.42211807,  614508.58356505, 1509064.71320334,
       2112284.64856071, 1862944.85549062, 2048062.25513506,
       1401753.6646986 ,

In [33]:
print("Best hyperparameters : ", gridsearch_lasso.best_params_)
print("Best R2 score : ", gridsearch_lasso.best_score_)

Best hyperparameters :  {'alpha': 30}
Best R2 score :  0.9161163117578266


We could compare the results from the two models but the R2 are quite comparable between lasso and ridge. Could use Lasso with the alpha=30 (best model) according to the gridsearch) to select the features with the most important coefficient and train the model again -> regularized

EDIT : Lasso already does that directly

Next step is to compare train./test and see if overfitting has been corrected

In [37]:
gridsearch_ridge.best_estimator_.score(X_train, y_train)

0.9687461008144985

In [38]:
print('Train score is : {:.2f}'.format(gridsearch_ridge.best_estimator_.score(X_train, y_train)))
print('Test score is : {:.2f}'.format(gridsearch_ridge.best_estimator_.score(X_test, y_test)))

Train score is : 0.97
Test score is : 0.93


In [39]:
print('Train score is : {:.2f}'.format(gridsearch_lasso.best_estimator_.score(X_train, y_train)))
print('Test score is : {:.2f}'.format(gridsearch_lasso.best_estimator_.score(X_test, y_test)))

Train score is : 0.97
Test score is : 0.94


In [40]:
'''
As reminder, here is the scores for baseline model
'''
print('Train score is : {:.2f}'.format(lr.score(X_train, y_train)))
print('Test score is : {:.2f}'.format(lr.score(X_test, y_test)))

Train score is : 0.97
Test score is : 0.94


So there is still overfitting, ridge and lasso should introduce more regularizaiton, maybe my grid search was not setup correctly ? 